|Variable|Variável     |Definição	                              |Chave                                         |
|--------|-------------|------------------------------------------|----------------------------------------------|
|survival|sobrevivência|Sobrevivência                             |0 = Não, 1 = Sim                              |
|pclass  |pclass	   |Classe de ingresso                        |1 = 1º, 2 = 2º, 3 = 3º                        |
|ex      |sexo         |Sexo                                      |                                              |
|Age     |Idade em anos| -                                        |                                              |
|sibsp   |sibsp        |Nº de irmãos / cônjuges a bordo do Titanic|	                                             |
|parch   |pergaminho   |Nº de pais / filhos a bordo do Titanic    |                                              |
|ticket  |bilhete      |Número do bilhete	                      |                                              |
|fare    |tarifa       |Tarifa de passageiro                      |	                                             |
|cabin   |cabine       |Número da cabine	                      |                                              |
|embarked|embarcou     |Porto de embarcação                       |C = Cherbourg, Q = Queenstown, S = Southampton|

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

In [66]:
combine = [train, test]

In [67]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

# pd.crosstab(train['Title'], train['Sex'])

In [69]:
train['Tick_num'] = train.Ticket.str.extract('\s([0-9]+)')
train[train['Ticket'] == 'LINE'] = -1
idx = train[train['Tick_num'].isna()].index
train['Tick_num'].loc[idx] = train['Ticket'].loc[idx]
train['Tick_num'] = train['Tick_num'].astype(int)

In [72]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Title           object
Tick_num         int64
dtype: object

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 
                                                'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()    

In [ ]:
title = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title)
    dataset['Title'] = dataset['Title'].fillna(0)

In [ ]:
# Convertendo em binario
for dataset in combine:
    transf = {"male": 0, "female": 1}
    dataset['Sex_bin'] = dataset['Sex'].map(transf).astype(int)

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    cab = {"S":1, "C":2, "Q":3}
    dataset['Embarked'] = dataset['Embarked'].map(cab).astype(int)

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

In [ ]:
for dataset in combine:
    idx = dataset['Age'].loc[dataset['Age']<1].index
    age = dataset['Age'].iloc[idx]*100
    dataset['Age'].iloc[idx] = age

    m = dataset['Age'].median()
    dataset['Age'] = dataset['Age'].fillna(m)

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
    dataset['Age'] = dataset['Age'].astype(int)

In [ ]:
for dataset in combine:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
# train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [ ]:
for dataset in combine:
    dataset.set_index('PassengerId', inplace=True)

In [ ]:
plt.scatter(train['Age'], train['Fare'])
plt.show()

### Modelos

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
train_num = train.select_dtypes(include=[np.number])

In [ ]:
test_num = test.select_dtypes(include=[np.number])

In [ ]:
x, y = train_num.drop('Survived', axis=1), train_num.Survived.copy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [ ]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
dectree_clf = DecisionTreeClassifier()

In [ ]:
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf),
                                         ('svc', svm_clf), ('dt', dectree_clf)], voting='hard')

In [ ]:
voting_clf.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
for clf in (log_clf,rnd_clf,svm_clf, dectree_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
ptest = voting_clf.predict(test_num)
prev = pd.Series(ptest, index= test.index, name='Survived')
prev.to_csv("modelo_ensemble_1.csv", header=True)    

In [ ]:
svm_clf.fit(x_train, y_train)

In [ ]:
ptest = svm_clf.predict(test_num)
prev = pd.Series(ptest, index= test.index, name='Survived')
prev.to_csv("modelo_svc_1.csv", header=True)    

In [ ]:
rnd_clf.fit(x_train, y_train)
ptest = rnd_clf.predict(test_num)
prev = pd.Series(ptest, index= test.index, name='Survived')
prev.to_csv("modelo_randoforest_1.csv", header=True)   